In [1]:
#@title Install meds-torch

!pip install meds-torch
!pip install hydra-joblib-launcher

  Using cached torch-2.5.1-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached lightning-2.4.0-py3-none-any.whl.metadata (38 kB)
  Using cached x_transformers-1.34.0-py3-none-any.whl.metadata (661 bytes)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached hydra_colorlog-1.2.0-py3-none-any.whl.metadata (949 bytes)
  Using cached torchmetrics-1.6.0-py3-none-any.whl.metadata (20 kB)
  Using cached torchvision-0.20.1-cp312-cp312-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached einops-0.8.0-py3-none-any.whl.metadata (12 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu1

In [ ]:
#@title Download E-ICU demo
import os
from pathlib import Path
notebook_dir = os.getcwd()

ROOT_DIR=f"{notebook_dir}/work_dir/mimiciv_demo/"
# ROOT_DIR=f"{notebook_dir}/work_dir/eicu_demo/"
Path(ROOT_DIR).mkdir(parents=True, exist_ok=True)

In [3]:
# Download pre-meds script, event config (defining how raw data is converted to meds data), and meds-transform config
!rm -rf {ROOT_DIR}/tmp/
!mkdir {ROOT_DIR}/meds_torch_example/
!git clone --depth 1 https://github.com/oufattole/meds-torch.git {ROOT_DIR}/tmp/
!mv {ROOT_DIR}/tmp/MIMICIV_INDUCTIVE_EXPERIMENTS {ROOT_DIR}/meds_torch_example

Cloning into '/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tmp'...
remote: Enumerating objects: 656, done.
remote: Counting objects: 100% (656/656), done.
remote: Compressing objects: 100% (473/473), done.
remote: Total 656 (delta 169), reused 442 (delta 112), pack-reused 0 (from 0)
Receiving objects: 100% (656/656), 705.59 KiB | 7.75 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [4]:
# Define the window sizes and aggregations to generate features for
MEDS_DIR = ROOT_DIR + "/meds/"
TASK_DIR = MEDS_DIR + "/task_labels"
TASK_NAME="mortality/in_icu/first_24h"
# TASK_NAME="los_in_hospital_first_48h"
OUTPUT_MODEL_DIR= ROOT_DIR + "/output/meds_tab/"

TUTORIAL_DIR= ROOT_DIR + "/meds_torch_example/MIMICIV_INDUCTIVE_EXPERIMENTS/"
CONFIGS_DIR=f"{TUTORIAL_DIR}/configs/"
TENSOR_DIR= ROOT_DIR + "/triplet_tensors/" # Output tensors directory
N_PARALLEL_WORKERS=1 # set to the number of parallel workers you want to use
PIPELINE_CONFIG_PATH=f"{CONFIGS_DIR}/triplet_config.yaml" # set to the directory in which the config file is stored, must be an absolute path.
JOBLIB_RUNNER_CONFIG_PATH=f"{CONFIGS_DIR}/joblib_runner.yaml" # set to the directory in which the config file is stored, must be an absolute path.

In [5]:
triplet_tensorization_config = """
defaults:
  - _preprocess
  - _self_
input_dir: ${oc.env:MEDS_DIR}
cohort_dir: ${oc.env:MODEL_DIR}
stages:
  - fit_filter_and_occlude
  - filter_measurements
  - filter_subjects
  - occlude_outliers
  - fit_normalization
  - fit_vocabulary_indices
  - normalization
  - tokenization
  - tensorization

stage_configs:
  filter_subjects:
    min_events_per_subject: 0
    min_measurements_per_subject: 0
  fit_filter_and_occlude:
    _script: MEDS_transform-aggregate_code_metadata
    aggregations:
      - code/n_occurrences
      - code/n_subjects
      - values/sum
      - values/sum_sqd
      - values/n_occurrences
  filter_measurements:
    min_subjects_per_code: 0
  fit_normalization:
    _script: MEDS_transform-aggregate_code_metadata
    aggregations:
      - code/n_occurrences
      - code/n_subjects
      - name: values/quantiles
        quantiles: [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
"""
from pathlib import Path

Path(PIPELINE_CONFIG_PATH).write_text(triplet_tensorization_config)

911

In [19]:
!bash "$TUTORIAL_DIR/tokenize.sh" "$MEDS_DIR" "$TENSOR_DIR" "$N_PARALLEL_WORKERS" "$PIPELINE_CONFIG_PATH" "stage_runner_fp=$JOBLIB_RUNNER_CONFIG_PATH"

Running extraction pipeline.
2024-12-15 03:05:23.970 | INFO     | MEDS_transforms.runner:main:326 - Running stage: fit_filter_and_occlude
2024-12-15 03:05:23.980 | INFO     | MEDS_transforms.runner:run_stage:255 - Running command: MEDS_transform-aggregate_code_metadata --config-dir=/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/meds_torch_example/MIMICIV_INDUCTIVE_EXPERIMENTS/configs --config-name=triplet_config --multirun 'hydra.searchpath=[pkg://MEDS_transforms.configs]' stage=fit_filter_and_occlude worker="range(0,1)" hydra/launcher=joblib
2024-12-15 03:05:26.269 | INFO     | MEDS_transforms.runner:run_stage:263 - Command output:
[2024-12-15 03:05:24,766][HYDRA] Joblib.Parallel(n_jobs=-1,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 1 jobs
[2024-12-15 03:05:24,766][HYDRA] Launching jobs, sweep output dir : /storage/nassim/projects/MEDS-DEV/demo/work_dir/

In [22]:
import polars as pl
metadata_df = pl.read_parquet(ROOT_DIR + "/triplet_tensors/metadata/codes.parquet")
metadata_df

code,code/n_occurrences,code/n_subjects,values/quantiles,values/sum,values/sum_sqd,values/n_occurrences,description,parent_codes,possibly_cpt_code,itemid,valueuom,code/vocab_index
str,u16,u8,struct[9],f32,f32,u16,str,list[str],list[str],list[str],list[str],u16
"""BMI (kg/m2)""",391,34,"{null,null,null,null,null,null,null,null,null}",0.0,0.0,0,null,null,null,null,null,1
"""Blood Pressure""",426,32,"{null,null,null,null,null,null,null,null,null}",0.0,0.0,0,null,null,null,null,null,2
"""Blood Pressure Lying""",1,1,"{null,null,null,null,null,null,null,null,null}",0.0,0.0,0,null,null,null,null,null,3
"""Blood Pressure Sitting""",1,1,"{null,null,null,null,null,null,null,null,null}",0.0,0.0,0,null,null,null,null,null,4
"""Blood Pressure Standing (3 min…",1,1,"{null,null,null,null,null,null,null,null,null}",0.0,0.0,0,null,null,null,null,null,5
…,…,…,…,…,…,…,…,…,…,…,…,…
"""TRANSFER_TO//transfer//Surgica…",6,6,"{null,null,null,null,null,null,null,null,null}",0.0,0.0,0,null,null,null,null,null,5573
"""TRANSFER_TO//transfer//Transpl…",11,4,"{null,null,null,null,null,null,null,null,null}",0.0,0.0,0,null,null,null,null,null,5574
"""TRANSFER_TO//transfer//Trauma …",8,7,"{null,null,null,null,null,null,null,null,null}",0.0,0.0,0,null,null,null,null,null,5575


In [31]:
TRAIN_OUTPUT_DIR = ROOT_DIR + "/meds_torch/train_output/"
train_cmd = f"""
meds-torch-train \
    experiment=triplet_mtr \
    data.dataloader.batch_size=4 \
    paths.data_dir={TENSOR_DIR} \
    paths.meds_cohort_dir={MEDS_DIR} \
    paths.output_dir={TRAIN_OUTPUT_DIR} \
    data.task_name={TASK_NAME} \
    data.task_root_dir={TASK_DIR} \
    hydra.searchpath=[pkg://meds_torch.configs,{CONFIGS_DIR}/meds-torch-configs]
"""
! $train_cmd

2024-12-15 03:32:01.416 | INFO     | meds_torch.utils.pylogger:info:88 - [rank0] Enforcing tags! <cfg.extras.enforce_tags=True>
2024-12-15 03:32:01.419 | INFO     | meds_torch.utils.pylogger:info:88 - [rank0] Printing config tree with Rich! <cfg.extras.print_config=True>
CONFIG
├── data
│   └── _target_: meds_torch.data.datamodule.MEDSDataModule.initialize          
│       dataset_cls: meds_torch.data.components.pytorch_dataset.PytorchDataset  
│       data_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//
│       meds_cohort_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv
│       cache_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/
│       code_metadata_fp: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimici
│       schema_files_root: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimic
│       tasks_root: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo
│       split_names:                                            

In [32]:
TRAIN_CKPT_PATH = !echo "$(meds-torch-latest-dir path={TRAIN_OUTPUT_DIR})/checkpoints/best_model.ckpt"
TRAIN_CKPT_PATH = TRAIN_CKPT_PATH[0]
print(TRAIN_CKPT_PATH)

/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds_torch/train_output/2024-12-15_03-32-01_385164/checkpoints/best_model.ckpt


In [34]:
PREDICT_OUTPUT_DIR = ROOT_DIR + "/meds_torch/predict_output/"
predict_cmd = f"""
meds-torch-predict \
    experiment=triplet_mtr \
    ckpt_path={TRAIN_CKPT_PATH} \
    paths.data_dir={TENSOR_DIR} \
    paths.meds_cohort_dir={MEDS_DIR} \
    paths.output_dir={PREDICT_OUTPUT_DIR} \
    data.task_name={TASK_NAME} \
    data.task_root_dir={TASK_DIR} \
    data.do_include_subject_id=True \
    data.do_include_prediction_time=True \
    hydra.searchpath=[pkg://meds_torch.configs,{CONFIGS_DIR}/meds-torch-configs]
"""
! $predict_cmd

2024-12-15 03:33:55.168 | INFO     | meds_torch.utils.pylogger:info:88 - [rank0] Enforcing tags! <cfg.extras.enforce_tags=True>
2024-12-15 03:33:55.173 | INFO     | meds_torch.utils.pylogger:info:88 - [rank0] Printing config tree with Rich! <cfg.extras.print_config=True>
2024-12-15 03:33:55.173 | WARNING  | meds_torch.utils.pylogger:warning:93 - [rank0] Field 'callbacks' not found in config. Skipping 'callbacks' config printing...
CONFIG
├── data
│   └── _target_: meds_torch.data.datamodule.MEDSDataModule.initialize          
│       dataset_cls: meds_torch.data.components.pytorch_dataset.PytorchDataset  
│       data_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//
│       meds_cohort_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv
│       cache_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/
│       code_metadata_fp: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimici
│       schema_files_root: /storage/nassim/projects/MEDS-DEV/de

In [37]:
PREDICT_OUTPUT_FP =!echo $(meds-torch-latest-dir path=$PREDICT_OUTPUT_DIR)/
PREDICT_OUTPUT_FP = PREDICT_OUTPUT_FP[0] + "/predict.parquet"
print(PREDICT_OUTPUT_FP)
predictions_df = pl.read_parquet(PREDICT_OUTPUT_FP)
predictions_df

/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds_torch/predict_output/2024-12-15_03-33-55_136346//predict.parquet


subject_id,prediction_time,boolean_value,predicted_boolean_value,predicted_boolean_probability,embeddings,logits
i64,datetime[ns],bool,bool,f64,list[f64],list[f64]
10002495,2141-05-23 20:18:01,false,false,0.136801,"[-2.545923, 0.025286, … 0.705734]",[-1.842114]
10004422,2111-01-18 09:44:50,false,false,0.124603,"[-2.531284, 0.059085, … 0.699216]",[-1.949547]
10004720,2186-11-13 19:55:00,true,false,0.113988,"[-2.485285, 0.113076, … 0.744316]",[-2.050634]
10004733,2174-12-05 11:28:24,false,false,0.109338,"[-2.510099, 0.121363, … 0.794356]",[-2.097525]
10007928,2129-04-07 00:25:00,false,false,0.116089,"[-2.485562, 0.094246, … 0.770296]",[-2.03]
…,…,…,…,…,…,…
10027445,2142-08-01 01:41:00,false,false,0.123286,"[-2.5613, 0.075122, … 0.75831]",[-1.96167]
10027602,2201-10-31 12:25:00,false,false,0.12781,"[-2.488924, 0.077882, … 0.719922]",[-1.92046]
10035631,2116-02-29 18:43:20,false,false,0.105775,"[-2.481711, 0.131169, … 0.778387]",[-2.134639]


In [38]:
from sklearn.metrics import roc_auc_score
roc_auc_score(predictions_df['boolean_value'], predictions_df['predicted_boolean_probability'])

np.float64(0.27499999999999997)

In [ ]:
# !pip install git+https://github.com/kamilest/meds-evaluation.git

  Cloning https://github.com/kamilest/meds-evaluation.git to /tmp/pip-req-build-7m5po6_6
  Running command git clone --filter=blob:none --quiet https://github.com/kamilest/meds-evaluation.git /tmp/pip-req-build-7m5po6_6
  Resolved https://github.com/kamilest/meds-evaluation.git to commit 472e15298e385ae148a48dcb3f3c31a67398ef94
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.2 MB/s eta 0:00:00
  Created wheel for meds-evaluation: filename=meds_evaluation-0.1.dev76+g472e152-py3-none-any.wh